# Matrix Encryption
Makenna Worley and Nic Van Der Werf

An exploration into encrypting data through matrices transformations and modular arithmetic

In [ ]:
# Importing and declaring global variables
import numpy as np
import math
import random

from ipywidgets import widgets
from IPython.display import display

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
# print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/our_super_secret_message.txt').read()

Functions that will help us with encrypting and decrypting our secret messages!

Generates a random 2 x 2 Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.

In [ ]:
def generate_key_matrix_2x2(modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(2, 2))

        # Calculating the determinant with Mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")

Inverts a 2 x 2 Matrix with the modulo using integers 

In [ ]:
def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo

Encrypts plain text using a 2 x 2 key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [ ]:
def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [ ]:
def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('*')
    return ''.join(plaintext_chars).rstrip('A')

Generates a random n x n Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.


In [ ]:
def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # All matrices must be invertible
            pass

Inverts a n x n matrix with the modulo using Gaussian Elimination

Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them

In [ ]:
def inverse_nxn_mod(matrix, modulo):
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo

Encrypts plain text using a n x n key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n key


In [ ]:
def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n inverse key


In [ ]:
def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')

Functions for creating and testing random strings with a 2 x 2 key matrix

In [ ]:
def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

First testing our matrix to make sure it is invertible and therefore useable for our purposes.

In [ ]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Now let's look at our encrypting a message using our predefined KEY_MATRIX_2x2

In [ ]:
MESSAGE = "Hello my name is Makenna"

inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)


Example with the generating of a 2 x 2 random key. 

In [ ]:
MESSAGE = "Hello my name is Nic"

key_matrix, inverse_key_matrix = generate_key_matrix_2x2(MODULO)

print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, key_matrix, MODULO, ALPHABET)

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)

In [ ]:
MESSAGE = text1
KEY_MATRIX_SIZE = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

In [ ]:
MESSAGE = text2
KEY_MATRIX_SIZE = 23

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

In [ ]:
MESSAGE = text3
KEY_MATRIX_SIZE = 100

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)